In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

BASE = "/content/drive/MyDrive/Ingrediants"  # <-- adapte si nécessaire
for root, dirs, files in os.walk(BASE):
    level = root.replace(BASE, '').count(os.sep)
    indent = ' ' * 2 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    for d in dirs:
        print(f"{indent}  - {d}")
    # n'affiche pas tous les fichiers pour garder lisible
    break


Ingrediants/
  - valid
  - train
  - Ingrediants
  - Copie de Ingrediants
  - output


In [ ]:
import os, json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

DRIVE_BASE = "/content/drive/MyDrive/Ingrediants"
OUTPUT_DIR = "/content/drive/MyDrive/Ingrediants/output"

os.makedirs(OUTPUT_DIR, exist_ok=True)
TRAIN_DIR = os.path.join(DRIVE_BASE, "train")
TEST_DIR  = os.path.join(DRIVE_BASE, "test")

BATCH_SIZE = 32
IMG_SIZE = (224,224)
EPOCHS = 12
LEARNING_RATE = 1e-4
MODEL_PATH = os.path.join(OUTPUT_DIR, "ingredient_model.h5")
LABELS_PATH = os.path.join(OUTPUT_DIR, "labels.json")

# === STOP SI MODELE EXISTE DEJA ===
if os.path.exists(MODEL_PATH):
    print("⚠️ Le modèle existe déjà, training ignoré :", MODEL_PATH)
    exit()   # Arrête le script

# ======================================================
# ============= ENTRAINEMENT UNIQUEMENT SI PAS DE MODELE =
# ======================================================

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
val_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1],3))
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base.input, outputs=preds)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
]

history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=callbacks)

base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=6, callbacks=callbacks)

model.save(MODEL_PATH)

labels = {v:k for k,v in train_gen.class_indices.items()}
with open(LABELS_PATH, 'w') as f:
    json.dump(labels, f)

print("Saved model to:", MODEL_PATH)
print("Saved labels to:", LABELS_PATH)


⚠️ Le modèle existe déjà, training ignoré : /content/drive/MyDrive/Ingrediants/output/ingredient_model.h5
Found 10594 images belonging to 49 classes.
Found 2624 images belonging to 49 classes.


KeyboardInterrupt: 

In [ ]:
import os

BASE = "/content/drive/MyDrive/Ingrediants/train"  # adapte si nécessaire
print(os.listdir(BASE))


['yellow onion', 'turnip', 'sugar', 'spinach', 'soy beans', 'sour cream', 'salt', 'raddish', 'purple onion', 'potatoes', 'plum tomatoes', 'peas', 'oil', 'lettuce', 'lemon', 'honey', 'ground turmeric', 'ground cinnamon', 'green onions', 'ginger', 'grated parmesan cheese', 'garlic powder', 'fresh parsley', 'garam masala', 'garlic', 'flour', 'eggplant', 'eggs', 'diced tomatoes', 'cumin', 'cucumber', 'corn starch', 'cilantro leaves', 'corn', 'chopped onion', 'chili powder', 'chicken', 'cauliflower', 'carrots', 'cabbage', 'capsicum', 'butter', 'brown sugar', 'black beans', 'black pepper', 'beetroot', 'bay leaves', 'avocado', 'baking powder']


In [ ]:
import os, shutil

paths = [
    "/content/drive/MyDrive/Ingrediants/train/Unlabeled",
    "/content/drive/MyDrive/Ingrediants/valid/Unlabeled",
    "/content/drive/MyDrive/Ingrediants/test/Unlabeled"  # optionnel
]

for p in paths:
    if os.path.exists(p):
        shutil.rmtree(p)
        print(f"Supprimé : {p}")
    else:
        print(f"Non trouvé : {p}")


Non trouvé : /content/drive/MyDrive/Ingrediants/train/Unlabeled
Non trouvé : /content/drive/MyDrive/Ingrediants/valid/Unlabeled
Non trouvé : /content/drive/MyDrive/Ingrediants/test/Unlabeled


In [ ]:
import os, json

TRAIN_DIR  = "/content/drive/MyDrive/Ingrediants/train"
LABELS_PATH = "/content/drive/MyDrive/Ingrediants/output/labels.json"

classes = sorted(os.listdir(TRAIN_DIR))
class_indices = {cls: idx for idx, cls in enumerate(classes)}

with open(LABELS_PATH, "w") as f:
    json.dump(class_indices, f)

print("labels.json propre généré ✔️")
print(class_indices)


labels.json propre généré ✔️
{'avocado': 0, 'baking powder': 1, 'bay leaves': 2, 'beetroot': 3, 'black beans': 4, 'black pepper': 5, 'brown sugar': 6, 'butter': 7, 'cabbage': 8, 'capsicum': 9, 'carrots': 10, 'cauliflower': 11, 'chicken': 12, 'chili powder': 13, 'chopped onion': 14, 'cilantro leaves': 15, 'corn': 16, 'corn starch': 17, 'cucumber': 18, 'cumin': 19, 'diced tomatoes': 20, 'eggplant': 21, 'eggs': 22, 'flour': 23, 'fresh parsley': 24, 'garam masala': 25, 'garlic': 26, 'garlic powder': 27, 'ginger': 28, 'grated parmesan cheese': 29, 'green onions': 30, 'ground cinnamon': 31, 'ground turmeric': 32, 'honey': 33, 'lemon': 34, 'lettuce': 35, 'oil': 36, 'peas': 37, 'plum tomatoes': 38, 'potatoes': 39, 'purple onion': 40, 'raddish': 41, 'salt': 42, 'sour cream': 43, 'soy beans': 44, 'spinach': 45, 'sugar': 46, 'turnip': 47, 'yellow onion': 48}


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import json
import os

TRAIN_DIR = "/content/drive/MyDrive/Ingrediants/train"
VAL_DIR   = "/content/drive/MyDrive/Ingrediants/valid"
MODEL_PATH = "/content/drive/MyDrive/Ingrediants/output/ingredient_model.h5"
LABELS_PATH = "/content/drive/MyDrive/Ingrediants/output/labels.json"

IMG_SIZE = (224, 224)
BATCH = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical'
)

# Enregistrer les labels
with open(LABELS_PATH, "w") as f:
    json.dump(train_gen.class_indices, f)

# Modèle MobileNetV2
base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(len(train_gen.class_indices), activation='softmax')(x)

model = Model(base.input, output)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=12
)

model.save(MODEL_PATH)
print("Modèle entraîné et sauvegardé ✔️")


Found 13218 images belonging to 49 classes.
Found 1646 images belonging to 49 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import json

st.title("📸 Détection d'Ingrédients en Temps Réel")

# Charger modèle
MODEL_PATH = "ingredient_model.h5"
LABEL_PATH = "labels.json"

model = tf.keras.models.load_model(MODEL_PATH)

# Charger labels
with open(LABEL_PATH, "r") as f:
    labels = json.load(f)

# Inverser (0: "onion", 1: "tomato", etc.)
labels = {v: k for k, v in labels.items()}

st.write("Prenez une photo et l’ingrédient sera identifié.")

img_input = st.camera_input("Prendre une photo")

if img_input:
    img = Image.open(img_input).resize((224,224))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)[0]
    idx = np.argmax(pred)
    confidence = float(pred[idx]) * 100
    ingredient = labels[idx]

    st.success(f"**Ingrédient détecté : {ingredient} ({confidence:.2f}%)**")


Overwriting app.py


In [ ]:
import shutil

shutil.copy(
    "/content/drive/MyDrive/Ingrediants/output/ingredient_model.h5",
    "/content/ingredient_model.h5"
)

shutil.copy(
    "/content/drive/MyDrive/Ingrediants/output/labels.json",
    "/content/labels.json"
)


'/content/labels.json'

In [ ]:
os.listdir("/content")


['.config',
 'log.txt',
 'ingredient_model.h5',
 'app.py',
 'labels.json',
 'drive',
 'ngrok.log',
 'sample_data']

In [ ]:
!pip install streamlit
!pip install pyngrok


In [ ]:
!pip install pyngrok
!ngrok version

ngrok version 3.34.0
pyngrok version 7.5.0


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("36WexiI6lJi0oOff5kSp3XA5k40_3roYcSGLEzx1WyjHTSNyt")
print("Authtoken OK ✔️")

Authtoken OK ✔️


In [ ]:
!streamlit run app.py &>/content/log.txt


In [ ]:
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "https://untippled-bettyann-unmortgageable.ngrok-free.dev" -> "http://localhost:8501">

In [ ]:
!ps -ef | grep streamlit

root       11962   11026  0 17:32 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root       11964   11962  0 17:32 ?        00:00:00 grep streamlit


In [ ]:
!pkill streamlit
!pkill ngrok

In [ ]:
!streamlit run app.py --server.port 8501 --server.headless true &>/content/log.txt &

In [ ]:
!ps -ef | grep streamlit

root       11988       1 84 17:33 ?        00:00:02 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 8501 --server.headless true
root       12003   11026  0 17:33 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root       12005   12003  0 17:33 ?        00:00:00 grep streamlit


In [ ]:
!ngrok config add-authtoken 36WexiI6lJi0oOff5kSp3XA5k40_3roYcSGLEzx1WyjHTSNyt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!curl -s http://localhost:4040/api/tunnels

In [ ]:
!ps -ef | grep -E "ngrok|streamlit"

root       11988       1 19 17:33 ?        00:00:02 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 8501 --server.headless true
root       12059   11026  0 17:33 ?        00:00:00 /bin/bash -c ps -ef | grep -E "ngrok|streamlit"
root       12061   12059  0 17:33 ?        00:00:00 grep -E ngrok|streamlit


In [ ]:
!pkill ngrok
!pkill streamlit

In [ ]:
!streamlit run app.py --server.port 8501 --server.headless true &>/content/log.txt &

In [ ]:
!ps -ef | grep streamlit


root       12089       1 21 17:33 ?        00:00:02 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 8501 --server.headless true
root       12132   11026  0 17:33 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root       12134   12132  0 17:33 ?        00:00:00 grep streamlit


In [ ]:
!ps -ef | grep ngrok

root       12147   11026  0 17:33 ?        00:00:00 /bin/bash -c ps -ef | grep ngrok
root       12149   12147  0 17:33 ?        00:00:00 grep ngrok


In [ ]:
!curl -s http://localhost:4040/api/tunnels

In [ ]:
!ngrok config add-authtoken 36WexiI6lJi0oOff5kSp3XA5k40_3roYcSGLEzx1WyjHTSNyt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!ngrok http 8501 --region=eu &>/content/ngrok.log &

In [ ]:
!ps -ef | grep ngrok


root       12194       1  4 17:33 ?        00:00:00 /usr/bin/python3 /usr/local/bin/ngrok http 8501 --region=eu
root       12195   12194  7 17:33 ?        00:00:00 /root/.config/ngrok/ngrok http 8501 --region=eu
root       12217   11026  0 17:33 ?        00:00:00 /bin/bash -c ps -ef | grep ngrok
root       12219   12217  0 17:33 ?        00:00:00 grep ngrok


In [ ]:
!curl -s http://localhost:4040/api/tunnels


{"tunnels":[{"name":"command_line","ID":"7dd69d83a8af918cd96b329e1b65ba22","uri":"/api/tunnels/command_line","public_url":"https://untippled-bettyann-unmortgageable.ngrok-free.dev","proto":"https","config":{"addr":"http://localhost:8501","inspect":true},"metrics":{"conns":{"count":13,"gauge":4,"rate1":0.004945678401958921,"rate5":0.009311412051549022,"rate15":0.007090980265504683,"p50":90488879390,"p90":197905770639.99994,"p95":251118605816,"p99":251118605816},"http":{"count":27,"rate1":0.01752399770746842,"rate5":0.022521487722436558,"rate15":0.14717849859693144,"p50":2290675,"p90":1861005609.8,"p95":1869549275,"p99":1870338663}}}],"uri":"/api/tunnels"}


In [ ]:
import pandas as pd

df = pd.read_csv("calories2.csv")


In [ ]:
!pkill -f streamlit

In [ ]:
!streamlit run app.py &>/content/log.txt &

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("36WexiI6lJi0oOff5kSp3XA5k40_3roYcSGLEzx1WyjHTSNyt")
ngrok.connect(8501)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The endpoint 'https://untippled-bettyann-unmortgageable.ngrok-free.dev' is already online. Either\n1. stop your existing endpoint first, or\n2. start both endpoints with `--pooling-enabled` to load balance between them.\r\n\r\nERR_NGROK_334\r\n"}}


In [ ]:
!ps -ef | grep streamlit

root       27420       1  1 18:36 ?        00:00:02 /usr/bin/python3 /usr/local/bin/streamlit run app.py
root       28001   11026  0 18:38 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root       28003   28001  0 18:38 ?        00:00:00 grep streamlit


In [ ]:
!pkill -f streamlit


In [ ]:
!streamlit run app.py &>/content/log.txt &

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("36WexiI6lJi0oOff5kSp3XA5k40_3roYcSGLEzx1WyjHTSNyt")
ngrok.connect(8501)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The endpoint 'https://untippled-bettyann-unmortgageable.ngrok-free.dev' is already online. Either\n1. stop your existing endpoint first, or\n2. start both endpoints with `--pooling-enabled` to load balance between them.\r\n\r\nERR_NGROK_334\r\n"}}
